In [1]:
import numpy as np

In [2]:
test_str = "D2FE28"

In [16]:
def hex2bin(c):
    return np.binary_repr(int(c, base=16), width=4)

In [18]:
def decode2bin(s):
    num = ""
    for c in s:
        num += hex2bin(c)
    return num

In [19]:
decode2bin(test_str)

'110100101111111000101000'